In [1]:
%load_ext autoreload
%autoreload 2

In [83]:
import sys
sys.path.append('../..')

from app.core.audio.AudioData import AudioData
from app.core.audio.AudioPlayer import AudioPlayer
from app.core.midi.MidiData import MidiData
from app.core.midi.MidiPlayer import MidiPlayer
from app.core.midi.MidiSynth import MidiSynth

from music21 import converter, note, expressions

In [121]:
import subprocess

def midi_to_lilypond(midi_file, output_ly_file):
    """
    Convert a MIDI file to a LilyPond (.ly) file using midi2ly.
    """
    subprocess.run(["midi2ly", midi_file, "-o", output_ly_file], check=True)

# Generate the LilyPond file from MIDI
midi_to_lilypond("fugue.mid", "fugue.ly")

LY output to `fugue.ly'...


In [147]:
import re

def add_arrows(ly_file, output_file, note_indices):
    """
    Adds an '↑' annotation above specific notes by index in a LilyPond (.ly) file,
    ensuring that annotations are only added after entering the note block.

    Args:
        ly_file (str): Path to the input LilyPond file.
        output_file (str): Path to save the output LilyPond file with annotations.
        note_indices (list of int): List of note indices to annotate.
    """
    note_indices = [n+1 for n in note_indices]
    
    with open(ly_file, 'r') as f:
        lines = f.readlines()

    # Refined regex to match only valid LilyPond notes
    note_pattern = re.compile(r"\b([a-g](?:['|,]*)\d*)\b")
    note_count = 0  # Track the current note index
    in_note_block = False  # Track if we're inside the note block
    annotated_lines = []

    for line in lines:
        # Check for the start of the note block
        if "{" in line and not in_note_block:
            in_note_block = True  # Start processing notes after the first '{'
            annotated_lines.append(line)
            continue
        
        # Skip lines that are comments or if not in a note block
        if line.strip().startswith("%") or not in_note_block:
            annotated_lines.append(line)
            continue

        # Define the annotation function to annotate specific notes
        def annotate(match):
            nonlocal note_count
            note = match.group(1)
            if note_count in note_indices:
                # Add arrow annotation using markup
                annotated_note = f"{note}^\\markup {{ \"↑\" }}"
            else:
                annotated_note = note
            note_count += 1
            return annotated_note

        # Apply the annotation function to each line containing notes
        annotated_line = note_pattern.sub(annotate, line)
        annotated_lines.append(annotated_line)

    # Write the annotated content to the output file
    with open(output_file, 'w') as f:
        f.writelines(annotated_lines)

    print(f"Annotations added to notes at indices {note_indices}. Saved to {output_file}")

# Usage example: Annotate the first, third, and fifth notes
indices_to_annotate = [0, 2, 4]
add_arrows("fugue.ly", "fugue_annot.ly", indices_to_annotate)

Annotations added to notes at indices [1, 3, 5]. Saved to fugue_annot.ly


In [162]:
import subprocess
import os

def ly_to_png(ly_file, output_dir=None):
    """
    Convert a LilyPond (.ly) file to a PNG image.

    Args:
        ly_file (str): Path to the input LilyPond file.
        output_dir (str, optional): Directory to save the PNG file. Defaults to the same directory as `ly_file`.

    Returns:
        str: Path to the generated PNG file.
    """
    # Use the directory of the input file if no output directory is specified
    if output_dir is None:
        output_dir = os.path.dirname(os.path.abspath(ly_file))  # Get absolute path of the ly_file's directory
    
    print(f"Output directory: {output_dir}")

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Run LilyPond to generate PNG, setting backend to EPS and resolution to 300 DPI for good quality
    try:
        subprocess.run([
            "lilypond",
            "-fpng",
            "-dresolution=600",
            ly_file  # Input .ly file
        ], check=True)

        # Find the generated PNG file
        base_name = os.path.splitext(os.path.basename(ly_file))[0]
        png_file = os.path.join(output_dir, f"{base_name}.png")
        
        if os.path.exists(png_file):
            print(f"Conversion successful! PNG saved as {png_file}")
            return png_file
        else:
            print("Error: PNG file not found. Conversion may have failed.")
            return None

    except subprocess.CalledProcessError:
        print("Error: LilyPond conversion failed. Ensure LilyPond is installed and in your PATH.")
        return None

# Example usage
ly_to_png(ly_file="fugue_annot.ly", output_dir=None)

Output directory: /Users/sarah/Desktop/shasha/virtuOS/notebooks/score


Processing `fugue_annot.ly'
Parsing...
fugue_annot.ly:22:9: error: wrong type for argument 2.  Expecting list of number pairs, found (make-music (quote TextScriptEvent) (quote direction) 1 (quote text) (markup #:line (#:simple "↑")))
  \key c
        ^\markup { "↑" } \major
fugue_annot.ly:22:26: warning: Ignoring non-music expression
  \key c^\markup { "↑" } 
                         \major

Note: compilation failed and \version outdated, did you
update input syntax with convert-ly?

  https://lilypond.org/doc/v2.24/Documentation/usage/updating-files-with-convert_002dly

Interpreting music...
Preprocessing graphical objects...
Interpreting music...
MIDI output to `fugue_annot.midi'...
Finding the ideal number of pages...
Fitting music on 1 page...
Drawing systems...
Converting to PNG...

Error: LilyPond conversion failed. Ensure LilyPond is installed and in your PATH.



fatal error: failed files: "fugue_annot.ly"


In [1]:
# Run the ScoreViewer
import sys
sys.path.append('..')

from ScoreViewer import RunScoreViewer
from PyQt6.QtWidgets import QApplication
from PyQt6.QtCore import QCoreApplication


if __name__ == '__main__':
    if not QCoreApplication.instance():
        app = QApplication(sys.argv)
    else:
        app = QCoreApplication.instance()

    score_viewer = RunScoreViewer(app, 'fugue_annot.png')
    

qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=446.349,321.874 gbl=446.349,321.874 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-446.349,-321.874 last=-446.349,-321.874 Δ 446.349,321.874) : no target window


In [118]:
from music21 import articulations
# parse XML into a score
score = converter.parse("fugue.musicxml")

for n in score.flat.notes:
    if isinstance(n, note.Note):
        n.style.color = 'red'
        exp = articulations.Staccato()
        n.expressions.append(exp)
        # print(n.expressions)

# Save the annotated score as MusicXML
score.write("musicxml", fp="fugue_annot.musicxml")

PosixPath('/Users/sarah/Desktop/shasha/virtuOS/notebooks/score/fugue_annot.musicxml')

In [119]:
import subprocess

def xml_to_png(xml_file, output_png):
    """
    Convert xml to png with musescore.

    Appends "-1", "-2", etc. for each page of score generated.
    Eg, output_png="fugue.png" -> "fugue-1.png"
    """
    # note: this works on my computer but may need to find more elegant
    # solution later on...
    MUSESCORE_PATH = "/Applications/MuseScore 3.app/Contents/MacOS/mscore"
    subprocess.run([MUSESCORE_PATH, "-o", output_png, xml_file], check=True)


xml_to_png("fugue_annot.musicxml", "fugue_annot.png")

dlopen error : dlopen(libjack.0.dylib, 0x0001): tried: 'libjack.0.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibjack.0.dylib' (no such file), '/Applications/MuseScore 3.app/Contents/Frameworks/libjack.0.dylib' (no such file), '/Applications/MuseScore 3.app/Contents/Frameworks/libjack.0.dylib' (no such file), '/usr/lib/libjack.0.dylib' (no such file, not in dyld cache), 'libjack.0.dylib' (no such file), '/usr/lib/libjack.0.dylib' (no such file, not in dyld cache) 
dlopen error : dlopen(/usr/local/lib/libjack.0.dylib, 0x0001): tried: '/usr/local/lib/libjack.0.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/lib/libjack.0.dylib' (no such file), '/usr/local/lib/libjack.0.dylib' (no such file) 
Creating main window…
ZoomBox::setLogicalZoom(): Formatting logical zoom level as 100% (rounded from 1.000000)
Reading translations…
convert <fugue_annot.musicxml>...
importMusicXml() file 'fugue_annot.musicxml' is not a valid MusicXML file
JIT is disabled for Q

In [63]:
# Convert xml to midi for analysis in app

score = converter.parse("fugue.musicxml")
score.write('midi', fp="fugue.mid")

# Create a synth with a soundfont
SOUNDFONT_FILEPATH = '../../app/resources/MuseScore_General.sf3'
midi_synth = MidiSynth(SOUNDFONT_FILEPATH)

# Load the midi file into a MidiData object
MIDI_FILEPATH = 'fugue.mid'
midi_data = MidiData(MIDI_FILEPATH)

# Create MidiSynth/Player objects
midi_player = MidiPlayer(midi_synth)
midi_player.load_midi(midi_data)

midi_player.play(start_time=0) # Play the MIDI

Loading MidiSynth...
Synth + soundfont loaded.


In [73]:
# turn xml into mido message format (unused)
from music21 import tempo, note
import mido

def parse_xml_to_mido_messages(xml_file_path: str) -> list:
    score = converter.parse(xml_file_path)
    midi_messages = []
    elapsed_time = 0  # Time in seconds

    # Extract tempo if it exists
    bpm = 100  # Default tempo if none is set
    score_bpm = score.flat.getElementsByClass(tempo.MetronomeMark)
    if score_bpm:
        bpm = score_bpm[0].number # take it as the first tempo if it exists

    sec_per_beat = 60 / bpm # convert beats/min -> sec/beat based on tempo

    # Traverse the score
    for part in score.parts:
        midi_channel = part.getInstrument().midiChannel if part.getInstrument() else 0
        
        # Handle program change
        instr = part.getInstrument()
        if instr:
            program_message = mido.Message('program_change', channel=midi_channel, program=instr.midiProgram)
            midi_messages.append(program_message)

        # Process each note or rest
        for elem in part.recurse():
            if isinstance(elem, note.Note) or isinstance(elem, note.Rest):
                start_time = elapsed_time + (elem.offset*sec_per_beat)
                duration_time = elem.quarterLength * sec_per_beat

                if isinstance(elem, note.Note):
                    # Create 'note_on' message
                    note_on_msg = mido.Message('note_on', channel=midi_channel, note=elem.pitch.midi, velocity=64, time=start_time)
                    midi_messages.append(note_on_msg)
                    
                    # Create 'note_off' message after the duration
                    note_off_msg = mido.Message('note_off', channel=midi_channel, note=elem.pitch.midi, velocity=64, time=start_time + duration_time)
                    midi_messages.append(note_off_msg)
                elif isinstance(elem, note.Rest):
                    # Treat rest as a silent period with just an elapsed time increment
                    elapsed_time += duration_time

    return midi_messages

midi_msgs = parse_xml_to_mido_messages("fugue.mxl")
# midi_msgs